# एजेन्टिक RAG विद् अटोजन

यो नोटबुकले अटोजन एजेन्टहरूको प्रयोग गरेर पुनःप्राप्ति-संवर्धित उत्पादन (RAG) कार्यान्वयन गर्ने र उन्नत मूल्याङ्कन क्षमताहरूको प्रदर्शन गर्दछ।


SQLite संस्करण समाधान  
यदि तपाईंले यो त्रुटि सामना गर्नुहुन्छ:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

तपाईंको नोटबुकको सुरुमा यो कोड ब्लक अनकमेन्ट गर्नुहोस्:  


In [2]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [3]:
import os
import time
import asyncio
from typing import List, Dict
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from azure.core.credentials import AzureKeyCredential
from autogen_ext.models.azure import AzureAIChatCompletionClient

import chromadb

load_dotenv()

True

## क्लाइन्ट बनाउनुहोस्

पहिले, हामी Azure AI Chat Completion Client लाई आरम्भ गर्छौं। यो क्लाइन्ट प्रयोगकर्ताका प्रश्नहरूको उत्तर उत्पन्न गर्न Azure OpenAI सेवासँग अन्तरक्रिया गर्न प्रयोग गरिन्छ।


In [4]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

## भेक्टर डाटाबेस सुरुवात

हामी ChromaDB लाई स्थायी भण्डारणको साथ सुरु गर्छौं र उन्नत नमूना कागजातहरू थप्छौं। ChromaDB लाई कागजातहरू भण्डारण र पुन: प्राप्त गर्न प्रयोग गरिन्छ, जसले सटीक उत्तरहरू उत्पन्न गर्न सन्दर्भ प्रदान गर्छ।


In [5]:
# Initialize ChromaDB with persistent storage
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.create_collection(
    name="travel_documents",
    metadata={"description": "travel_service"},
    get_or_create=True
)

# Enhanced sample documents
documents = [
    "Contoso Travel offers luxury vacation packages to exotic destinations worldwide.",
    "Our premium travel services include personalized itinerary planning and 24/7 concierge support.",
    "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.",
    "Popular destinations include the Maldives, Swiss Alps, and African safaris.",
    "Contoso Travel provides exclusive access to boutique hotels and private guided tours."
]

# Add documents with metadata
collection.add(
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))],
    metadatas=[{"source": "training", "type": "explanation"} for _ in documents]
)

## सन्दर्भ प्रदायक कार्यान्वयन

`ContextProvider` वर्गले विभिन्न स्रोतहरूबाट सन्दर्भ प्राप्ति र एकीकरणको जिम्मा लिन्छ:

1. **भेक्टर डाटाबेस प्राप्ति**: यात्रा सम्बन्धी कागजातहरूमा सेमान्टिक खोज गर्न ChromaDB प्रयोग गर्दछ  
2. **मौसम जानकारी**: प्रमुख शहरहरूको लागि सिमुलेटेड मौसम डाटाबेस राख्छ  
3. **एकीकृत सन्दर्भ**: कागजात र मौसम दुवै डाटालाई समग्र सन्दर्भमा संयोजन गर्दछ  

मुख्य विधिहरू:
- `get_retrieval_context()`: सोधपुछको आधारमा सम्बन्धित कागजातहरू प्राप्त गर्दछ  
- `get_weather_data()`: निर्दिष्ट स्थानहरूको लागि मौसम जानकारी प्रदान गर्दछ  
- `get_unified_context()`: कागजात र मौसम सन्दर्भलाई संयोजन गरी उन्नत प्रतिक्रियाहरूका लागि प्रयोग गर्दछ  


In [6]:
class ContextProvider:
    def __init__(self, collection):
        self.collection = collection
        # Simulated weather database
        self.weather_database = {
            "new york": {"temperature": 72, "condition": "Partly Cloudy", "humidity": 65, "wind": "10 mph"},
            "london": {"temperature": 60, "condition": "Rainy", "humidity": 80, "wind": "15 mph"},
            "tokyo": {"temperature": 75, "condition": "Sunny", "humidity": 50, "wind": "5 mph"},
            "sydney": {"temperature": 80, "condition": "Clear", "humidity": 45, "wind": "12 mph"},
            "paris": {"temperature": 68, "condition": "Cloudy", "humidity": 70, "wind": "8 mph"},
        }
    
    def get_retrieval_context(self, query: str) -> str:
        """Retrieves relevant documents from vector database based on query."""
        results = self.collection.query(
            query_texts=[query],
            include=["documents", "metadatas"],
            n_results=2
        )
        context_strings = []
        if results and results.get("documents") and len(results["documents"][0]) > 0:
            for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
                context_strings.append(f"Document: {doc}\nMetadata: {meta}")
        return "\n\n".join(context_strings) if context_strings else "No relevant documents found"
    
    def get_weather_data(self, location: str) -> str:
        """Simulates retrieving weather data for a given location."""
        if not location:
            return ""
            
        location_key = location.lower()
        if location_key in self.weather_database:
            data = self.weather_database[location_key]
            return f"Weather for {location.title()}:\n" \
                   f"Temperature: {data['temperature']}°F\n" \
                   f"Condition: {data['condition']}\n" \
                   f"Humidity: {data['humidity']}%\n" \
                   f"Wind: {data['wind']}"
        else:
            return f"No weather data available for {location}."
    
    def get_unified_context(self, query: str, location: str = None) -> str:
        """Returns a unified context combining both document retrieval and weather data."""
        retrieval_context = self.get_retrieval_context(query)
        
        weather_context = ""
        if location:
            weather_context = self.get_weather_data(location)
            weather_intro = f"\nWeather Information for {location}:\n"
        else:
            weather_intro = ""
        
        return f"Retrieved Context:\n{retrieval_context}\n\n{weather_intro}{weather_context}"

## एजेन्ट कन्फिगरेसन

हामी पुनःप्राप्ति र सहायक एजेन्टहरू कन्फिगर गर्छौं। पुनःप्राप्ति एजेन्टले सेम्यान्टिक खोज प्रयोग गरेर सान्दर्भिक जानकारी फेला पार्नमा विशेषज्ञता राख्छ, जबकि सहायकले पुनःप्राप्त जानकारीको आधारमा विस्तृत उत्तरहरू उत्पन्न गर्छ।


In [7]:
# Create agents with enhanced capabilities
assistant = AssistantAgent(
    name="assistant",
    model_client=client,
    system_message=(
        "You are a helpful AI assistant that provides answers using ONLY the provided context. "
        "Do NOT include any external information. Base your answer entirely on the context given below."
    ),
)

## RAG प्रयोग गरेर प्रश्न प्रक्रिया

हामी `ask_rag` नामको फङ्सन परिभाषित गर्छौं जसले सहायकलाई प्रश्न पठाउँछ, उत्तर प्रक्रिया गर्छ, र यसको मूल्याङ्कन गर्छ। यो फङ्सनले सहायकसँगको अन्तरक्रिया व्यवस्थापन गर्छ र उत्तरको गुणस्तर मापन गर्न मूल्याङ्कनकर्ता प्रयोग गर्छ।


In [8]:
async def ask_rag_agent(query: str, context_provider: ContextProvider, location: str = None):
    """
    Sends a query to the assistant agent with context from the provider.
    
    Args:
        query: The user's question
        context_provider: The context provider instance
        location: Optional location for weather queries
    """
    try:
        # Get unified context
        context = context_provider.get_unified_context(query, location)
        
        # Augment the query with context
        augmented_query = (
            f"{context}\n\n"
            f"User Query: {query}\n\n"
            "Based ONLY on the above context, please provide a helpful answer."
        )

        # Send the augmented query to the assistant
        start_time = time.time()
        response = await assistant.on_messages(
            [TextMessage(content=augmented_query, source="user")],
            cancellation_token=CancellationToken(),
        )
        processing_time = time.time() - start_time
        
        return {
            'query': query,
            'response': response.chat_message.content,
            'processing_time': processing_time,
            'location': location
        }
    except Exception as e:
        print(f"Error processing query: {e}")
        return None

# उदाहरण प्रयोग

हामी evaluator लाई आरम्भ गर्छौं र ती प्रश्नहरू परिभाषित गर्छौं जसलाई हामी प्रक्रिया गर्न र मूल्याङ्कन गर्न चाहन्छौं।


In [ ]:
async def main():
    # Initialize context provider
    context_provider = ContextProvider(collection)
    
    # Example queries
    queries = [
        {"query": "What does Contoso's travel insurance cover?"},
        {"query": "What's the weather like in London?", "location": "london"},
        {"query": "What luxury destinations does Contoso offer and what's the weather in Paris?", "location": "paris"},
    ]
    
    print("=== Autogen RAG Demo ===")
    for query_data in queries:
        query = query_data["query"]
        location = query_data.get("location")
        
        print(f"\n\nQuery: {query}")
        if location:
            print(f"Location: {location}")
        
        # Show the context being used
        context = context_provider.get_unified_context(query, location)
        print("\n--- Context Used ---")
        print(context)
        print("-------------------")
        
        # Get response from the agent
        result = await ask_rag_agent(query, context_provider, location)
        if result:
            print(f"\nResponse: {result['response']}")
        print("\n" + "="*50)

## स्क्रिप्ट चलाउनुहोस्

हामी जाँच गर्छौं कि स्क्रिप्ट अन्तरक्रियात्मक वातावरणमा चलिरहेको छ वा साधारण स्क्रिप्टमा, र त्यसअनुसार मुख्य कार्य सञ्चालन गर्छौं।


In [12]:
if __name__ == "__main__":
    if asyncio.get_event_loop().is_running():
        await main()
    else:
        asyncio.run(main())

=== Autogen RAG Demo ===


Query: What does Contoso's travel insurance cover?

--- Context Used ---
Retrieved Context:
Document: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


-------------------

Response: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.



Query: What's the weather like in London?
Location: london

--- Context Used ---
Retrieved Context:
Document: Popular destinations include the Maldives, Swiss Alps, and African safaris.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


Weather Information for london:
Weather for London:
T


---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादहरूमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छन्। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्त्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।  
